<a href="https://colab.research.google.com/github/AFalabella0/SpeechBrain/blob/master/HyperPyYaml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HyperPyYAML Tutorial

An essential part of any deep learning pipeline is the definition of hyperparameters and other metadata. These data in conjunction with the deep learning algorithms control the various aspects of the pipeline, such as model architecture, training, and decoding. At SpeechBrain, we decided that the distinction between hyperparameters and learning algorithms ought to be evident in the structure of our toolkit, so we split our recipes into two primary files: `train.py` and `train.yaml`.

The `train.yaml` file is in a SpeechBrain-developed format, which we call "HyperPyYAML". We chose to extend YAML since it is a highly readable format for data serialization. By extending an already useful format, we were able to create an expanded definition of hyperparameter, keeping our actual experimental code small and highly readable.

Here's a short example with PyTorch code to motivate the use of HyperPyYAML, though PyTorch is not a requirement in order to use HyperPyYAML:

In [ ]:
%%capture
!pip install torch
!pip install hyperpyyaml

In [ ]:
import torch
from hyperpyyaml import load_hyperpyyaml

example_hyperparams = """
base_channels: 32
kernel_size: 11
padding: !ref <kernel_size> // 2

layer1: !new:torch.nn.Conv1d
  in_channels: 1
  out_channels: !ref <base_channels>
  kernel_size: !ref <kernel_size>
  padding: !ref <padding>

layer2: !new:torch.nn.Conv1d
  in_channels: !ref <base_channels>
  out_channels: !ref <base_channels> * 2
  kernel_size: !ref <kernel_size>
  padding: !ref <padding>

layer3: !new:torch.nn.Conv1d
  in_channels: !ref <base_channels> * 2
  out_channels: 1
  kernel_size: !ref <kernel_size>
  padding: !ref <padding>

model: !new:torch.nn.Sequential
  - !ref <layer1>
  - !new:torch.nn.LeakyReLU
  - !ref <layer2>
  - !new:torch.nn.LeakyReLU
  - !ref <layer3>
"""

# Create model directly by loading the YAML
loaded_hparams = load_hyperpyyaml(example_hyperparams)
model = loaded_hparams["model"]

# Transform a 2-second audio clip
input_audio = torch.rand(1, 1, 32000)
transformed_audio = model(input_audio)
print(transformed_audio.shape)

# Try a different hyperparameter value by overriding the padding value
loaded_hparams = load_hyperpyyaml(example_hyperparams, {"padding": 0})
model = loaded_hparams["model"]
transformed_audio = model(input_audio)
print(transformed_audio.shape)

torch.Size([1, 1, 32000])
torch.Size([1, 1, 31970])


As this example shows, HyperPyYAML allows for complex hyperparameter definitions with compositions. In addition, any value can be overridden for hyperparameter tuning. To grasp how all of this works, let's first briefly look at the basics of YAML.

## Basic YAML syntax

Enough prelude: lets talk YAML! Here's a brief example of a yaml snippet and what it would look like once loaded to python:

In [ ]:
import yaml
yaml_string = """
foo: 1
bar:
  - item1
  - item2
baz:
  item1: 3.4
  item2: True
"""
yaml.safe_load(yaml_string)

{'bar': ['item1', 'item2'], 'baz': {'item1': 3.4, 'item2': True}, 'foo': 1}

As you can see, YAML has built-in support for a variety of data types, including string, int, float, bool, list, and dictionary. Our HyperPyYAML format keeps all of this functionality.

In [ ]:
from hyperpyyaml import load_hyperpyyaml
load_hyperpyyaml(yaml_string)

{'bar': ['item1', 'item2'], 'baz': {'item1': 3.4, 'item2': True}, 'foo': 1}

Our primary additions to yaml format are added with YAML tags. Tags are added before an item definition, and are prefixed with `!`. For the purpose of illustrating how tags are used, here is an example with a minor addition that we've made, the `!tuple` tag:

In [ ]:
yaml_string = """
foo: !tuple (3, 4)
"""
load_hyperpyyaml(yaml_string)

{'foo': (3, 4)}

Now you know the YAML basics, time to move on to our additions!

## Tags `!new:` and `!name:`
YAML tags can contain a suffix to more specifically define what type of tag it is. We use this to define a tag that is able to create any python object, not just a basic type. This tag starts with `!new:` and contains the type of the object. For example:

In [ ]:
yaml_string = """
foo: !new:collections.Counter
"""
loaded_yaml = load_hyperpyyaml(yaml_string)
loaded_yaml["foo"]

Counter()

In [ ]:
loaded_yaml["foo"].update({"a": 3, "b": 5})
loaded_yaml["foo"]["a"] += 1
loaded_yaml["foo"]

Counter({'a': 4, 'b': 5})

Of course many python objects take arguments during creation. These arguments can be passed with a list for positional arguments or a dictionary for keyword arguments.

In [ ]:
yaml_string = """
foo: !new:collections.Counter
  - [a, b, r, a, c, a, d, a, b, r, a]
bar: !new:collections.Counter
  a: 2
  b: 1
  c: 5
"""
load_hyperpyyaml(yaml_string)

{'bar': Counter({'a': 2, 'b': 1, 'c': 5}),
 'foo': Counter({'a': 5, 'b': 2, 'c': 1, 'd': 1, 'r': 2})}

Another python object that is useful to create is a function object. In HyperPyYAML this can be done with the `!name:` tag. Behind the scenes, this tag uses `functools.partial` to create a new function definition with the default arguments provided. For example:

In [ ]:
yaml_string = """
foo: !name:collections.Counter
  a: 2
"""
loaded_yaml = load_hyperpyyaml(yaml_string)
loaded_yaml["foo"](b=4)

Counter({'a': 2, 'b': 4})

The default arguments can be overridden, just as a normal python function

In [ ]:
loaded_yaml["foo"](a=3, b=5)

Counter({'a': 3, 'b': 5})

## Tags `!ref` and `!copy`

Of course some hyperparameters get used in multiple places, so we added a mechanism for referring to another item called `!ref`. The node that this tag is applied to must be a string that contains the location of the node to copy. Sub-nodes can be accessed with square brackets, same as in Python. For example:

In [ ]:
yaml_string = """
foo:
  a: 3
  b: 4
bar:
  c: !ref <foo>
  d: !ref <foo[b]>
"""
load_hyperpyyaml(yaml_string)

{'bar': {'c': {'a': 3, 'b': 4}, 'd': 4}, 'foo': {'a': 3, 'b': 4}}

The `!ref` tag can support simple arithmetic and string concatenation for basic hyperparameter combinations.

In [ ]:
yaml_string = """
folder1: abc/def
folder2: ghi/jkl
folder3: !ref <folder1>/<folder2>

foo: 1024
bar: 512
baz: !ref <foo> // <bar> + 1
"""
load_hyperpyyaml(yaml_string)

{'bar': 512,
 'baz': 3,
 'folder1': 'abc/def',
 'folder2': 'ghi/jkl',
 'folder3': 'abc/def/ghi/jkl',
 'foo': 1024}

The `!ref` tag can also refer to objects, in which case it makes a reference to the same object, rather than a copy. If you'd prefer to make a copy instead, use the `!copy` tag.

In [ ]:
yaml_string = """
foo: !new:collections.Counter
  a: 4
bar: !ref <foo>
baz: !copy <foo>
"""
loaded_yaml = load_hyperpyyaml(yaml_string)
loaded_yaml["foo"].update({"b": 10})
print(loaded_yaml["bar"])
print(loaded_yaml["baz"])

Counter({'b': 10, 'a': 4})
Counter({'a': 4})


## Other tags

We introduced a variety of other tags as well:
* `!tuple` to create python tuples. Note this is implicitly resolved, so you do not need to explicitly write out the tuple tag, just use parantheses as you would in Python.
* `!include` to insert other yaml files directly
* `!apply` to load and execute a python function, storing the result

We use `!apply` to set the random seed at the beginning of loading the yaml, so that the models have the same parameters each run. The result is not stored, because it starts with `__`.

In [ ]:
yaml_string = """
sum: !apply:sum
  - [1, 2]
__set_seed: !apply:torch.manual_seed [1234]
"""
load_hyperpyyaml(yaml_string)

{'sum': 3}

## Overrides

In order to run experiments with various values for a hyperparameter, we have a system for overriding the values that are listed in the yaml file.

In [ ]:
overrides = {"foo": 7}
fake_file = """
foo: 2
bar: 5
"""
load_hyperpyyaml(fake_file, overrides)

{'bar': 5, 'foo': 7}

As shown in this example, overrides can take an ordinary python dictionary. However, this form does not support python objects. To override a python object, overrides can also take a yaml-formatted string with the HyperPyYAML syntax.

In [ ]:
load_hyperpyyaml(fake_file, "foo: !new:collections.Counter")

{'bar': 5, 'foo': Counter()}

## Conclusion

We are proud to present our HyperPyYAML syntax, and we think that it provides a readable and concise way to structure hyperparameter definitions. In addition, it removes unnecessary complexity from experiment files, allowing the algorithms to become clear. As is evident in the first example, overrides are easy, making hyperparameter tuning a cinch. Overall, we have found this package to be a valuable tool for deep learning!